# Notice
## This python code is an unofficial Tensorflow implementation of the model from following reference
### Authors : Havaei,M.; Davy,A.; Warde-Farley,D.; Biard,A.; Courville,A.; Bengio,Y.; Pal,C.; Jodoin,P.M.; Larochelle,H. 
### Title : Brain tumor segmentation with Deep Neural Networks
### Source : Med.Image Anal., 2017, 35, 18-31, Elsevier B.V, Netherlands
###### The code has not been completed yet. 

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

In [2]:
import numpy as np
import random
import json
import h5py
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
from skimage import io, color, img_as_float
from skimage.exposure import adjust_gamma
from skimage.segmentation import mark_boundaries
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.metrics import classification_report

In [3]:
import tensorflow as tf

In [4]:
import SimpleITK as sitk
from ipywidgets import interact, FloatSlider
from myshow import myshow, myshow3d
from TwoPathCNN import vanilla_CNN
from BatchDatasetReader import BatchDataset

In [5]:
class Cascaded_architecture(vanilla_CNN, object):
    def CNN_for_INPUT_Cascade(self, x):
        cascadedLayer = self.Defining_CNN(x, 65)
        y_conv = self.Defining_CNN(x, 33, ("InputCas", cascadedLayer))
        return y_conv
    
    def CNN_for_LOCAL_Cascade(self, x):
        cascadedLayer = self.Defining_CNN(x, 56)
        y_conv = self.Defining_CNN(x, 33, ("LocalCas", cascadedLayer))
        return y_conv
    
    def CNN_for_MF_Cascade(self, x):
        cascadedLayer = self.Defining_CNN(x, 53)
        y_conv = self.Defining_CNN(x, 33, ("MFCas", cascadedLayer))
        return y_conv
        
        

In [ ]:
def Feeding_Data_and_Realizing_the_Model(_):
    # data를 import한다.
    img_T1 = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T1.54567/VSD.Brain.XX.O.MR_T1.54567.mha")
    img_T2 = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T2.54569/VSD.Brain.XX.O.MR_T2.54569.mha")
    img_FLAIR = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_Flair.54566/VSD.Brain.XX.O.MR_Flair.54566.mha")
    img_T1c = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T1c.54568/VSD.Brain.XX.O.MR_T1c.54568.mha")
    img_GT = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain_3more.XX.O.OT.54571/VSD.Brain_3more.XX.O.OT.54571.mha")

    # 모델을 생성한다.
    img_T1_Slice = img_T1[:,:,77]
    img_T1c_Slice = img_T1c[:,:,77]
    img_T2_Slice = img_T2[:,:,77]
    img_FLAIR_Slice = img_FLAIR[:,:,77]
    img_GT_Slice = img_GT[:,:,77]
    imgVolume = sitk.JoinSeries(img_T1_Slice, img_T1c_Slice, img_T2_Slice, img_FLAIR_Slice)
    width = imgVolume.GetWidth()
    height = imgVolume.GetHeight()
    depth = imgVolume.GetDepth()
    X = tf.placeholder(tf.float32, shape=[None, width*height*depth])
    ### NOTE: If you call this function often, you may want to (i) move the `np.pad()`
    #### into the graph as `tf.pad()`, and (ii) replace `X_t` with a placeholder.
    ###X = np.pad(X, (pad_amount, 0), 'constant')
    ###X_t  = tf.convert_to_tensor(X)
    ###output_t = tf.map_fn(window_func, tf.range(window_size))
    ###output = sess.run(output_t)

    
    y_ = tf.placeholder(tf.float32, [None, width*height])

    # Deep Neural Networks 그래프를 생성한다.
    Graph = Cascaded_architecture()
    y_conv = Graph.CNN_for_INPUT_Cascade(X)
    
    # loss와 optimizer를 정의한다.  ###############################################

    # Cross Entropy를 비용함수(loss function)으로 정의하고, AdamOptimizer를 이용해서 비용 함수를 최소화한다.
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
  
    # 정확도를 측정한다.
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    



## Hyper-parameters

In [ ]:
FLAGS = tf.flags.FLAGS
tf.flags.DEFINE_integer("batch_size", 1, "batch size for training")
tf.flags.DEFINE_string("logs_dir", "logs/", "path to logs directory")
tf.flags.DEFINE_string("data_dir", "Data_zoo/", "path to dataset")
tf.flags.DEFINE_float("learning_rate", 1e-4, "Learning rate for Adam Optimizer")
tf.flags.DEFINE_string("model_dir", "Model_zoo/", "Path to vgg model mat")
tf.flags.DEFINE_bool('debug', "False", "Debug mode: True/ False")
tf.flags.DEFINE_string('mode', "train", "Mode train/ test/ valid_visualize/ test_visualize")

#MAX_ITERATION = int(1e5 + 1)
MAX_ITERATION = int(10000 + 1)
NUM_OF_CLASSESS = 5
IMAGE_SIZE = 224

## Batch manager

In [ ]:
def main(argv=None):
    keep_probability = tf.placeholder(tf.float32, name="keep_probabilty")
    image = tf.placeholder(tf.float32, shape=[None, IMAGE_SIZE, IMAGE_SIZE, 3], name="input_image")
    annotation = tf.placeholder(tf.int32, shape=[None, IMAGE_SIZE, IMAGE_SIZE, 1], name="annotation")

    pred_annotation, logits = inference(image, keep_probability)
    tf.summary.image("input_image", image, max_outputs=2)
    tf.summary.image("ground_truth", tf.cast(annotation, tf.uint8), max_outputs=2)
    tf.summary.image("pred_annotation", tf.cast(pred_annotation, tf.uint8), max_outputs=2)
    loss = tf.reduce_mean((tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                                          labels=tf.squeeze(annotation, squeeze_dims=[3]),
                                                                          name="entropy")))
    tf.summary.scalar("entropy", loss)

    trainable_var = tf.trainable_variables()
    if FLAGS.debug:
        for var in trainable_var:
            utils.add_to_regularization_and_summary(var)
    train_op = train(loss, trainable_var)

    print("Setting up summary op...")
    summary_op = tf.summary.merge_all()

    print("Setting up image reader...")
    train_records, valid_records, test_records = scene_parsing.read_dataset(FLAGS.data_dir)
    print(len(train_records))
    print(len(valid_records))
    print(len(test_records))

    print("Setting up dataset reader")
    image_options = {'resize': True, 'resize_size': IMAGE_SIZE}
    if FLAGS.mode == 'train':
        train_dataset_reader = BatchDataset(train_records, image_options)
    validation_dataset_reader = BatchDataset(valid_records, image_options)
    test_dataset_reader = BatchDataset(test_records, image_options)
      
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40
    sess = tf.Session(config=config)

    print("Setting up Saver...")
    saver = tf.train.Saver()
    summary_writer = tf.summary.FileWriter(FLAGS.logs_dir, sess.graph)

    sess.run(tf.global_variables_initializer())
    ckpt = tf.train.get_checkpoint_state(FLAGS.logs_dir)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print("Model restored...")

    if FLAGS.mode == "train":
        for itr in xrange(MAX_ITERATION):
            train_images, train_annotations = train_dataset_reader.next_batch(FLAGS.batch_size)
            feed_dict = {image: train_images, annotation: train_annotations, keep_probability: 0.85}

            sess.run(train_op, feed_dict=feed_dict)

            if itr % 10 == 0:
                train_loss, summary_str = sess.run([loss, summary_op], feed_dict=feed_dict)
                print("Step: %d, Train_loss:%g" % (itr, train_loss))
                summary_writer.add_summary(summary_str, itr)

            if itr % 500 == 0:
                valid_images, valid_annotations = validation_dataset_reader.next_batch(FLAGS.batch_size)
                valid_loss = sess.run(loss, feed_dict={image: valid_images, annotation: valid_annotations,
                                                       keep_probability: 1.0})
                print("%s ---> Validation_loss: %g" % (datetime.datetime.now(), valid_loss))
                saver.save(sess, FLAGS.logs_dir + "model.ckpt", itr)

    elif FLAGS.mode == "valid_visualize":
        print('validation dataset visualization')
        valid_images, valid_annotations = validation_dataset_reader.get_random_batch(FLAGS.batch_size)
        pred = sess.run(pred_annotation, feed_dict={image: valid_images, annotation: valid_annotations,
                                                    keep_probability: 1.0})
        valid_annotations = np.squeeze(valid_annotations, axis=3)
        pred = np.squeeze(pred, axis=3)

        for itr in range(FLAGS.batch_size):
            utils.save_image(valid_images[itr].astype(np.double), FLAGS.logs_dir, name="inp_" + str(5+itr))
            utils.save_image(valid_annotations[itr].astype(np.double), FLAGS.logs_dir, name="gt_" + str(5+itr))
            utils.save_image(pred[itr].astype(np.double), FLAGS.logs_dir, name="pred_" + str(5+itr))
            print("Saved image: %d" % itr)

    elif FLAGS.mode == "test_visualize":
        print('test dataset visualization')
        test_images, test_annotations = test_dataset_reader.get_random_batch(FLAGS.batch_size)
        pred = sess.run(pred_annotation, feed_dict={image: test_images, annotation: test_annotations,
                                                    keep_probability: 1.0})
        test_annotations = np.squeeze(test_annotations, axis=3)
        pred = np.squeeze(pred, axis=3)

        for itr in range(FLAGS.batch_size):
            utils.save_image(test_images[itr].astype(np.double), FLAGS.logs_dir, name="inp_" + str(5+itr))
            utils.save_image(test_annotations[itr].astype(np.double), FLAGS.logs_dir, name="gt_" + str(5+itr))
            utils.save_image(pred[itr].astype(np.double), FLAGS.logs_dir, name="pred_" + str(5+itr))
            print("Saved image: %d" % itr)